# T1046 - Network Service Scanning
Adversaries may attempt to get a listing of services running on remote hosts, including those that may be vulnerable to remote software exploitation. Methods to acquire this information include port scans and vulnerability scans using tools that are brought onto a system. 

Within cloud environments, adversaries may attempt to discover services running on other cloud hosts. Additionally, if the cloud environment is connected to a on-premises environment, adversaries may be able to identify services running on non-cloud systems as well.

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/haresudhan/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - Port Scan
Scan ports to check for listening ports.

Upon successful execution, sh will perform a network connection against a single host (192.168.1.1) and determine what ports are open in the range of 1-65535. Results will be via stdout.

**Supported Platforms:** linux, macos
#### Attack Commands: Run with `sh`
```sh
for port in {1..65535};
do
  echo >/dev/tcp/192.168.1.1/$port && echo "port $port is open" || echo "port $port is closed" : ;
done
```

In [ ]:
Invoke-AtomicTest T1046 -TestNumbers 1

### Atomic Test #2 - Port Scan Nmap
Scan ports to check for listening ports with Nmap.

Upon successful execution, sh will utilize nmap, telnet, and nc to contact a single or range of adresseses on port 80 to determine if listening. Results will be via stdout.

**Supported Platforms:** linux, macos
#### Dependencies:  Run with `sh`!
##### Description: Check if nmap command exists on the machine

##### Check Prereq Commands:
```sh
if [ -x "$(command -v nmap)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
echo "Install nmap on the machine to run the test."; exit 1;

```

In [ ]:
Invoke-AtomicTest T1046 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
nmap -sS 192.168.1.0/24 -p 80
telnet 192.168.1.1 80
nc -nv 192.168.1.1 80
```

In [ ]:
Invoke-AtomicTest T1046 -TestNumbers 2

### Atomic Test #3 - Port Scan NMap for Windows
Scan ports to check for listening ports for the local host 127.0.0.1
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: NMap must be installed

##### Check Prereq Commands:
```powershell
if (cmd /c "nmap 2>nul") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest -OutFile $env:temp\nmap-7.80-setup.exe https://nmap.org/dist/nmap-7.80-setup.exe
Start-Process $env:temp\nmap-7.80-setup.exe /S

```

In [ ]:
Invoke-AtomicTest T1046 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
nmap 127.0.0.1```

In [ ]:
Invoke-AtomicTest T1046 -TestNumbers 3

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Lateral Movement, based on the information obtained.

Normal, benign system and network events from legitimate remote service scanning may be uncommon, depending on the environment and how they are used. Legitimate open port and vulnerability scanning may be conducted within the environment and will need to be deconflicted with any detection capabilities developed. Network intrusion detection systems can also be used to identify scanning activity. Monitor for process use of the networks and inspect intra-network flows to detect port scans.

# Shield Active Defense
## Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
### Use Case
A defender can change the output of a recon commands to hide simulation elements you don’t want attacked and present simulation elements you want the adversary to engage with.
### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.